In [1]:
# if colab

# !pip install pybullet
# !pip install gym
# !apt-get install python-opengl -y
# !apt install xvfb -y
# !pip install gym pyvirtualdisplay > /dev/null 2>&1
# !pip install -q git+https://github.com/tensorflow/examples.git

In [2]:
import os
import cv2
import tensorflow as tf 
from tensorflow.keras import layers, models
import numpy as np 
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
import pybullet_envs
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

In [3]:
np_seed = 654765645
tf_seed = 776644345
np.random.seed(np_seed)
tf.random.set_seed(tf_seed)

# check if GPU
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# colab

# from google.colab import drive
# drive.mount('/content/drive')

# root_dir = "drive/My Drive/"
# base_dir = root_dir + 'CPCtesting'
# os.makedirs(base_dir,exist_ok=True)

# train_dir = base_dir + '/train'
# os.makedirs(train_dir,exist_ok=True)

# model_dir = base_dir + '/model'
# os.makedirs(model_dir,exist_ok=True)

# if local machine
base_dir = "."

train_dir = base_dir + '/train'
os.makedirs(train_dir,exist_ok=True)

model_dir = base_dir + '/model'
os.makedirs(model_dir,exist_ok=True)

logs_base_dir = base_dir + '/logs'
# tensorboard
%load_ext tensorboard
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [5]:
# get data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [6]:
class CPCModel(tf.keras.Model):
    def __init__(self,code_size, predict_terms, terms=4, units=256, image_size=64, channels=3):
        super(CPCModel, self).__init__()
        self.code_size = code_size
        self.predict_terms = predict_terms
        self.terms = terms
        self.units = units
        self.image_size = image_size
        self.channels = channels

        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.lrelu1 = tf.keras.layers.LeakyReLU()
        self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.lrelu2 = tf.keras.layers.LeakyReLU()
        self.conv3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.lrelu3 = tf.keras.layers.LeakyReLU()
        self.conv4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
        self.bn4 = tf.keras.layers.BatchNormalization()
        self.lrelu4 = tf.keras.layers.LeakyReLU()
        self.flatten = tf.keras.layers.Flatten()
        self.dense5 = tf.keras.layers.Dense(units=256, activation='linear')
        self.bn5 = tf.keras.layers.BatchNormalization()
        self.lrelu5 = tf.keras.layers.LeakyReLU()
        self.dense6 = tf.keras.layers.Dense(units=code_size, activation='linear', name='encoder_embedding')

        self.gru = tf.keras.layers.GRU(units, return_sequences=False, name='ar_context')
        self.linear = tf.keras.layers.Dense(predict_terms*code_size, activation='linear')    
   
    def encoding(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.lrelu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.lrelu2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.lrelu3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.lrelu4(x)
        x = self.flatten(x)
        x = self.dense5(x)
        x = self.bn5(x)
        x = self.lrelu5(x)
        z = self.dense6(x)
        return z
  
    def get_context(self, x):
        z = self.encoding(x)
        z = tf.reshape(z, [-1, self.terms, self.code_size])
        c = self.gru(z)
        return c
    def get_prediction(self, x):
        c = self.get_context(x)
        z_hats = self.linear(c)
        z_hat = tf.reshape(z_hats, [-1, self.predict_terms, self.code_size])
        return z_hat

    def optimizer(self):
        pass

    def loss(self,weights,biases,labels,inputs,num_samples,num_classes): 
        loss = tf.nn.nce_loss(
        weights, biases, labels, inputs, num_sampled, num_classes, num_true=1,
        sampled_values=None, remove_accidental_hits=False, name='nce_loss')
        return loss
  
    def call(self,inputs):
        x_tm, x_tp = inputs
        x_tm = tf.reshape(x_tm, [-1, self.image_size, self.image_size, self.channels])
        x_tp = tf.reshape(x_tp, [-1, self.image_size, self.image_size, self.channels])
        z_hat = self.get_prediction(x_tm)
        z_tp = self.encoding(x_tp)
        z_tp = tf.reshape(z_tp, [-1, self.predict_terms, self.code_size])
        dot_prods = tf.reduce_mean(tf.reduce_mean(z_hat*z_tp, axis=-1), axis=-1, keepdims=True)
        probs = tf.sigmoid(dot_prods)
        return probs


  # def save(self):
  #       f1 = os.path.join(folder,'target_actor')
  #       f2 = os.path.join(folder, 'target_critic')
  #       f3 = os.path.join(folder, 'actor')
  #       f4 = os.path.join(folder, 'critic')
  #       self.target_actor.save(f1)
  #       self.target_critic.save(f2)
  #       self.actor.save(f3)
  #       self.critic.save(f4)


  # def load(self):
  #   pass

In [7]:
class ReplayBuffer():
    def __init__(self,state_space,action_space,capacity,batch):
        self.capacity = capacity
        self.batch = batch
        
        self.avaliable_batch = 0
        self.idx = 0
        self.entries = 0 
        
        self.states = np.empty((self.capacity,state_space),dtype = np.float32)
        self.next_states = np.empty((self.capacity,state_space),dtype = np.float32)
        self.actions = np.empty((self.capacity,action_space),dtype = np.float32)
        self.rewards = np.empty((self.capacity,1),dtype = np.float32)
        self.not_dones = np.empty((self.capacity, 1), dtype=np.float32)
        
    def add(self,state,next_state,action,reward,done):
        np.copyto(self.states[self.idx], state)
        np.copyto(self.actions[self.idx], action)
        np.copyto(self.rewards[self.idx], reward)
        np.copyto(self.next_states[self.idx], next_state)
        np.copyto(self.not_dones[self.idx], not done)
        self.avaliable_batch= (self.avaliable_batch + 1) if self.avaliable_batch < self.batch else self.batch
        self.entries = (self.entries + 1) if self.entries < self.capacity else self.capacity
        self.idx = (self.idx + 1) % self.capacity
        
    def sample(self):
        num = self.avaliable_batch
        if(num > self.batch):
            num = self.batch
        print('avaliable_batch: ',self.avaliable_batch, "entries: ", self.entries,'capacity: ', self.capacity)
        idx = np.random.choice(self.avaliable_batch,size = num,replace=False)
        
        states = tf.convert_to_tensor(self.states[idx])
        next_states = tf.convert_to_tensor(self.next_states[idx])
        actions = tf.convert_to_tensor(self.actions[idx])
        rewards = tf.convert_to_tensor(self.rewards[idx])
        not_dones = tf.convert_to_tensor(self.not_dones[idx])
        
        return states,next_states,actions,rewards,not_dones
        
        

class Actor(tf.keras.Model):
    def __init__(self,action_space,critic,actor_lr = 0.001):
        super(Actor,self).__init__()
        
        #self.critic = critics
        
        #optimizer
        self.opt = tf.keras.optimizers.Adam(actor_lr)
        self.critic = critic
       
        #model
        self.dense1 = tf.keras.layers.Dense(400,activation = 'relu',dtype='float32')
        self.dense2 = tf.keras.layers.Dense(300,activation='relu',dtype='float32')
        self.dense3 = tf.keras.layers.Dense(action_space,activation = 'tanh',dtype='float32')    
        
    def loss(self,states,actions):
        actions = self(states)
        #stateactions = tf.concat([states,actions],-1)
        Q = self.critic(states,actions)
        loss = - tf.reduce_mean(Q)
        return loss
    
    def update(self,states,actions):
        with tf.GradientTape() as tape:
            loss = self.loss(state,state)

        grad = tape.gradient(loss,self.trainable_variables)
        self.opt.apply_gradients(zip(grad, self.trainable_variables))
        #print('actor loss: ', loss ,"\n" )
        return loss
    
    def call(self,x):
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

class Critic(tf.keras.Model):
    def __init__(self,critic_lr = 0.001):
        super(Critic,self).__init__()
        
        # optimizer
        self.opt = tf.keras.optimizers.Adam(critic_lr)
        
        # loss
        #self.loss = tf.keras.losses.MSE
        
        
        # layers
        self.dense1 = tf.keras.layers.Dense(400,activation = 'relu',dtype='float32')
        self.dense2 = tf.keras.layers.Dense(300,activation='relu',dtype='float32')
        self.dense3 = tf.keras.layers.Dense(1,dtype='float32') 
        
    #loss
    def loss(self,actual,pred):
        return tf.keras.losses.MSE(actual,pred)
    
    def update(self,states,actions,Q_h):
        with tf.GradientTape() as tape:
            Q = self.call(states,actions)
            loss = self.loss(Q,Q_h)

        grad = tape.gradient(loss,self.trainable_variables)
        self.opt.apply_gradients(zip(grad, self.trainable_variables))
        #print('critic loss: ', loss ,"\n" )
        return loss
    
    #predict
    def call(self,states,actions):
        x = tf.concat([states,actions],-1)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x
    
    
class SAC(tf.keras.Model):
    def __init__(self,state_space,action_space,capacity = 100,batch = 1, tau=0.999,gamma=0.9,actor_lr = 0.01, critic_lr = 0.001):
        super(SAC,self).__init__()
        # tensorboard callbacks
        self.cb = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=1/3, patience=2, min_lr=1e-4),
                   tf.keras.callbacks.ModelCheckpoint('weights/weights.{epoch:02d}-{val_binary_accuracy:.2f}.cpkt',
                                          monitor='val_binary_accuracy', save_best_only=True, save_weights_only=True),
                   tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3),
                   tf.keras.callbacks.TensorBoard()]
        
        
        #hyperparameters
        self.batch = batch
        self.tau = tau
        self.gamma = gamma
        self.actor_lr = actor_lr
        self.critic_lr = critic_lr
        
        
        #spaces
        self.action_space = action_space
        self.state_space = state_space
        self.state_action_space = self.action_space + self.state_space
        
        # replay buffer
        self.replay_buffer = ReplayBuffer(self.state_space,self.action_space,capacity,self.batch)
        
        # models
        self.critic = Critic(critic_lr = critic_lr)        
        self.actor = Actor(self.action_space,actor_lr=actor_lr,critic = self.critic)
        self.critic.compile(optimizer = self.critic.opt,loss = self.critic.loss)
        self.actor.compile(optimizer = self.actor.opt,loss = self.actor.loss)
        
        # target models
        self.target_actor = Actor(self.action_space,actor_lr=actor_lr,critic = self.critic)
        self.target_critic = Critic(critic_lr = critic_lr)  
        self.target_actor.set_weights(self.actor.get_weights())
        self.target_critic.set_weights(self.critic.get_weights())
        
        #cpc
        #self.cpc = CPC(code_size=128, predict_terms=4, terms=4, units=256, image_size=64, channels=3)
    
    def store_replay(self,state,next_state,action,reward,done):
        self.replay_buffer.add(state,next_state,action,reward,done)
    
    def set_labels(self,states,new_states,actions,rewards):
        mu = self.target_actor(new_states)
        #print(mu,states)
#         stateactions = tf.concat([states,mu],1)
        self.Q_h = self.target_critic(new_states,mu)
        y = rewards + self.gamma * self.Q_h
        #y = np.concatenate(self.y,0).astype('float32') #.reshape((self.minibatch_size,1,1,1))
        #print('y: ',self.y)
        y = tf.reshape(y,(self.replay_buffer.avaliable_batch,1,1,1))
        return y 
    
    def update_target_weights(self):   
        tgt_critic_weight = self.target_critic.get_weights()
        tgt_actor_weight = self.target_actor.get_weights()
        actor_weight = self.actor.get_weights()
        critic_weight = self.target_actor.get_weights()
        
        for idx,(part_tgt,part_net) in enumerate(zip(tgt_critic_weight,critic_weight)):
            tgt_critic_weight[idx] = self.tau*part_tgt + (1-self.tau)*part_net
        
        for idx,(part_tgt,part_net) in enumerate(zip(tgt_actor_weight,actor_weight)):
            tgt_actor_weight[idx] = self.tau*part_tgt + (1-self.tau)*part_net
            
    def save(self,filename):
        pass
    
    def load(self,filename):
        pass
            

In [8]:
class DataHandler:
    def __init__(self, batch_size, terms, predict_terms=1, image_size=64, color=False, rescale=True, aug=True, is_training=True, method='cpc'):
        self.batch_size = batch_size
        self.terms = terms
        self.predict_terms = predict_terms
        self.image_size = image_size
        self.color = color
        self.rescale = rescale
        self.aug = aug
        self.is_training = is_training
        self.method = method
        self.lena = cv2.imread(os.path.join(base_dir,'lena.jpg'))
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
        if self.is_training:
            self.x = x_train/255.0
            self.y = y_train
        else:
            self.x = x_test/255.0
            self.y = y_test
        self.idxs = []
        for i in range(10):
            y = y_train if self.is_training else y_test
            self.idxs.append(np.where(y == i)[0])
        self.n_samples = len(self.y)//terms if self.method == 'cpc' else len(self.y)
        self.shape = self.x.shape
        self.n_batches = self.n_samples//batch_size

    def __iter__(self):
        return self

    def __next__(self):
        return self.cpc_batch() if self.method == 'cpc' else self.benchmark_batch()

    def __len__(self):
        return self.n_batches

    def cpc_batch(self):
        img_labels = np.zeros((self.batch_size, self.terms + self.predict_terms))
        sentence_labels = np.ones((self.batch_size, 1)).astype('int32')
        for bi in range(self.batch_size):
            seed = np.random.randint(10)
            sentence = np.arange(seed, seed + self.terms + self.predict_terms) % 10
            if bi < self.batch_size//2:
                num = np.arange(10)
                predicted = sentence[-self.predict_terms:]
                for i, p in enumerate(predicted):
                    predicted[i] = np.random.choice(num[num != p], 1)
                sentence[-self.predict_terms:] = predicted % 10
                sentence_labels[bi, :] = 0
            img_labels[bi, :] = sentence
        images = self.get_samples(img_labels).reshape((self.batch_size, self.terms+self.predict_terms, self.image_size, self.image_size, 3))
        x_images = images[:, :-self.predict_terms, ...]
        y_images = images[:, -self.predict_terms:, ...]
        idx = np.random.choice(self.batch_size, self.batch_size, replace=False)
        return [x_images[idx], y_images[idx]], sentence_labels[idx]

    def get_samples(self, img_labels):
        idx = []
        for label in img_labels.flatten():
            idx.append(np.random.choice(self.idxs[int(label)], 1)[0])
        img_batch = self.x[idx, :, :]
        if self.aug:
            img_batch = self._aug_batch(img_batch)
        return img_batch

    def _aug_batch(self, img_batch):
        if self.image_size != 28:
            resized = []
            for i in range(img_batch.shape[0]):
                resized.append(cv2.resize(img_batch[i], (self.image_size, self.image_size)))
            img_batch = np.stack(resized)
        img_batch = img_batch.reshape((img_batch.shape[0], 1, self.image_size, self.image_size))
        img_batch = np.concatenate([img_batch, img_batch, img_batch], axis=1)

        if self.color:
            img_batch[img_batch >= 0.5] = 1
            img_batch[img_batch < 0.5] = 0
            for i in range(img_batch.shape[0]):
                x_c = np.random.randint(0, self.lena.shape[0] - self.image_size)
                y_c = np.random.randint(0, self.lena.shape[1] - self.image_size)
                img = self.lena[x_c:x_c+self.image_size, y_c:y_c+self.image_size]
                img = np.array(img).transpose((2, 0, 1))/255.0
                for j in range(3):
                    img[j, :, :] = (img[j, :, :] + np.random.uniform(0, 1))/2.0
                img[img_batch[i, :, :, :] == 1] = 1 - img[img_batch[i, :, :, :] == 1]
                img_batch[i, :, :, :] = img

        if self.rescale:
            img_batch = img_batch * 2 - 1
        img_batch = img_batch.transpose((0, 2, 3, 1))
        return img_batch

    def benchmark_batch(self):
        idx = np.random.choice(len(self.x), self.batch_size, replace=False)
        img_batch = self.x[idx]
        label_batch = self.y[idx]
        if self.aug:
            img_batch = self._aug_batch(img_batch)
        label_batch = label_batch.reshape((-1, 1))
        return img_batch, label_batch

In [9]:
# #train loop
# dh_train = DataHandler(64, 4, predict_terms=4, image_size=64, color=True, rescale=True, aug=True, is_training=True, method='cpc')
# dh_test = DataHandler(64, 4, predict_terms=4, image_size=64, color=True, rescale=True, aug=True, is_training=False, method='cpc')
# accuracy_metric_train = tf.keras.metrics.BinaryAccuracy()
# loss_metric_train = tf.keras.metrics.BinaryCrossentropy()
# accuracy_metric_test = tf.keras.metrics.BinaryAccuracy()
# loss_metric_test = tf.keras.metrics.BinaryCrossentropy()
# cpc = CPCModel(code_size=128, predict_terms=4, terms=4, units=256, image_size=64, channels=3)
# optim = tf.keras.optimizers.Adam(1e-3)
# cb = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=1/3, patience=2, min_lr=1e-4),
#       tf.keras.callbacks.ModelCheckpoint('weights/weights.{epoch:02d}-{val_binary_accuracy:.2f}.cpkt',
#                                           monitor='val_binary_accuracy', save_best_only=True, save_weights_only=True),
#       tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3),
#       tf.keras.callbacks.TensorBoard()]
# cpc.compile(optimizer=optim, loss='binary_crossentropy', metrics=['binary_accuracy'])
# cpc.fit(x=dh_train, epochs=10, validation_data=dh_test, steps_per_epoch=60000//64, validation_steps=10000//64, callbacks=cb)


In [10]:
%tensorboard --logdir {logs_base_dir}

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [11]:
# train loop params


episodes = 1
episode_steps = 200

# pybullet setup
env = gym.make('HalfCheetahBulletEnv-v0')
env.render(mode = 'human')
env._max_episode_steps = episode_steps
env = gym.wrappers.Monitor(env, "baseline_training", video_callable=lambda episode: True, force="true")



C:\Users\dtara\anaconda3\envs\csci-7000-rl\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [12]:
#network batch
batch = 16


#get spaces
state_space = env.observation_space.shape[0]
action_space = env.action_space.shape[0]
print(state_space,action_space)

26 6


In [13]:
state = env.reset()
sac = SAC(action_space=action_space,state_space=state_space,capacity = 100,batch = 32)

for episode in range(episodes):
    sumreward = 0
    for step in range(episode_steps):
        #print(observation)
        print('t: ',step, ' :episode: ',episode )
        #print('state: ',state)
        
        # get action
        state = tf.cast(tf.reshape(state,(1,1,state_space)),dtype='float32')
        #print(state)
        tensor_action = sac.actor(state)
        action = tensor_action[0][0]
        #print('action: ',action)
        
        # execute action
        next_state, reward, done, info = env.step(action)
        sumreward += reward

        # store transitions
        sac.store_replay(state,next_state,action,reward,done)
        
        #print('state: ',state)
        #print('next_state: ',next_state)
        #print('action: ',action)
        #print('reward: ',reward)

        #sample minibatch from data
        states,next_states,actions,rewards,not_done = sac.replay_buffer.sample()
        
        #set labels y_i
        y = sac.set_labels(states,next_states,actions,rewards)
        
        # update critic net
        sac.critic.update(states, actions, y)
        #losses[episode*timesteps + t] = loss
        #losses[i_episode*timesteps+] = history.history
        
        #update actor net
        sac.actor.update(states,actions)
        #print('weight check: ',rl.actor.get_weights(),'\n')
        
        #update target nets
        sac.update_target_weights()
        
        state = next_state
        if done:
            state = env.reset()
            #rewards[episode] = sumreward
            sac.save('baseline')
            print("Episode {} finished after {} timesteps with reward {}".format(episode,step+1,sumreward))
            break
print('done') 
sac.save('baseline')

t:  0  :episode:  0
avaliable_batch:  1 entries:  1 capacity:  100
t:  1  :episode:  0
avaliable_batch:  2 entries:  2 capacity:  100
t:  2  :episode:  0
avaliable_batch:  3 entries:  3 capacity:  100
t:  3  :episode:  0
avaliable_batch:  4 entries:  4 capacity:  100
t:  4  :episode:  0
avaliable_batch:  5 entries:  5 capacity:  100
t:  5  :episode:  0
avaliable_batch:  6 entries:  6 capacity:  100
t:  6  :episode:  0
avaliable_batch:  7 entries:  7 capacity:  100
t:  7  :episode:  0
avaliable_batch:  8 entries:  8 capacity:  100
t:  8  :episode:  0
avaliable_batch:  9 entries:  9 capacity:  100
t:  9  :episode:  0
avaliable_batch:  10 entries:  10 capacity:  100
t:  10  :episode:  0
avaliable_batch:  11 entries:  11 capacity:  100
t:  11  :episode:  0
avaliable_batch:  12 entries:  12 capacity:  100
t:  12  :episode:  0
avaliable_batch:  13 entries:  13 capacity:  100
t:  13  :episode:  0
avaliable_batch:  14 entries:  14 capacity:  100
t:  14  :episode:  0
avaliable_batch:  15 entrie

avaliable_batch:  32 entries:  100 capacity:  100
t:  117  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  118  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  119  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  120  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  121  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  122  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  123  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  124  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  125  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  126  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  127  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  128  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  129  :episode:  0
avaliable_batch:  32 entries:  100 capacity:  100
t:  130  :epis

 https://datascience.stackexchange.com/questions/13216/intuitive-explanation-of-noise-contrastive-estimation-nce-loss(InfoNCE Loss )
<br>
Representation Learning with Contrastive Predictive Coding
<br>
https://github.com/gdao-research/cpc/blob/master/cpc/data_handler.py (CPC)
<br>
https://github.com/davidtellez/contrastive-predictive-coding/blob/master/train_model.py (CPC)
<br>
https://github.com/MishaLaskin/curl/blob/23b0880708c29b078b0a25e62ff31fb587587b18/utils.py#L123 (replay buffer and SAC)
<br>
https://github.com/marload/DeepRL-TensorFlow2/blob/master/A2C/A2C_Discrete.py (A2C)
<br>
https://github.com/germain-hug/Deep-RL-Keras/blob/master/A3C/a3c.py (A3C)